

# 코드 구성
- 설정 - 라이브러리 로드 - 사용할 함수 정의 - 데이터 전처리 및 분리 - 모델 구성 - 훈련 - 추론

# 요약
## 설정
- "./vocab", "./model" 폴더가 코드 실행에 필요합니다.
- "./dataset/" 폴더에 데이콘에서 제공한 데이터가 있어야합니다.

## 필요한 함수 정의
- seed_everything
    - 재현성을 위한 함수
- text_preprocessing 
    - ip 패턴(ex 127.0.0.1) 혹은 ip.port(ex 127.0.0.1.8888) 를 모두 "IP"로 치환.
    - 영어와 한글만 남김
- return_vocab
    - vocabulary 생성 함수
    - 문장을 space단위로 분리 후 vocab을 생성
- word2int
    - word단위 token을 정수로 치환하여 정수 시퀀스를 만듦
- padding
    - 같은 길이로 만들어 주기 위해 뒷부분을 padding(0)으로 채움
    - max_length(최대 길이)를 해당 노트북에서는 200으로 함.
- return_similarity
    - 텍스트 유사도 계산
- level
    - threshold 값 이상인 값이 없으면 level 7 반환
    
## 전처리
- 처음 데이터를 볼 때 비정상적으로 긴 길이의 log인 452419를 제외 하였음. 하지만 영어와 한글만 남기는 전처리를 거치면 사용해도 상관 없을것으로 보임
- 치환, 영어와 한글만 남기는 전처리 후 중복되는 데이터는 하나만 남김.
- level 2,4,6은 샘플 수가 저기 떄문에 중복되는 텍스트라도 제거하지 않고 validation성능 측정 때 사용되도록 함.

## 텍스트 유사도
- 숫자를 제외한 영어와 한글만 남겨 전처리 했을 때 한 단어 차이로 level이 다른 샘플이 존재함.
- 제공된 데이터 전처리한 결과들끼리 자카드 유사도를 구하여 0.5이상인데 level이 다르면 무조건 훈련 샘플로 사용할 수 있도록 해당 index를 저장해둠
- 60818 * 60181 번 loop가 실행되기 떄문에 시간이 꽤 오래 걸림.

## 훈련/검증 데이터 분리
- 5-fold로 분리
- 각 폴드에서 텍스트 유사도가 높았던 index는 훈련샘플에는 넣고 검증 샘플에서는 제거.

## Multiple-Kernel Size CNNs-BiLSTM
- 각각 다른 kernel size를 가진 Conv를 통해 시퀀스에서 패턴을 추출하고 LSTM을 통해 순서정보를 학습하는 것을 기대하고 모델을 구성하였습니다.
- 전처리된 정수 시퀀스가 모델에 입력.(x)
- 입력된 정수 시퀀스는 Embedding Layer를 통해 각 정수들은 1024 차원으로 인코딩되어 벡터 시퀀스로 변환.(nn.Embedding)
- 벡터 시퀀스는 kernel size가 다른 6개의 Conv로 입력(nn.Conv1d)되고 출력된 feature map은 Maxpooling.(nn.MaxPool1d)
- 6개의 CNN에서 출력된 값들을 이어준 후 (torch.cat) BatchNorm을 적용.
- BiLSTM에 입력(nn.LSTM)되고 출력된 값들을 average pooling 한후 squeeze 해줌.
- 마지막 출력은 class 개수(7)로 출력.

## Training 
- 5개 폴드
- Epoch : 30
- batchSize : 400
- optimizer : adam
- scheduler : CosineAnnealingLR
- loss : CrossEntropy
- save : validation f1-macro
- 훈련시 confidence가 0.85 이상인 것이 없으면 7로 분류하여 성능평가하였음.

## Test inference
- 훈련 데이터와 같은 방법으로 전처리. vocab은 훈련데이터에서 만든 vocab을 사용.
- 5-fold 훈련을 했지만 validation 성능이 좋지 않았던 4번 째 fold로 만든 모델은 test inference로 사용하지 않음.
- 4개 모델로 예측한 값들을 평균내어 confidence가 0.7 이상인 것이 없으면 level 7로 반환 하였음.

# =============================================================

# 설정
- "./vocab", "./model" 폴더가 필요
- 데이콘에서 제공된 데이터셋은 "./dataset" 에 있음.

In [1]:
# !mkdir vocab
# !mkdir model

In [2]:
pip install torch_poly_lr_decay

ERROR: Could not find a version that satisfies the requirement torch_poly_lr_decay (from versions: none)
ERROR: No matching distribution found for torch_poly_lr_decay
Note: you may need to restart the kernel to use updated packages.


# 라이브러리 로드

In [4]:
import warnings
warnings.filterwarnings('ignore')
import math
import pickle
import pandas as pd
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import re
from nltk.tokenize import word_tokenize
import torch.nn as nn
from torch.nn import functional as F

from torch.utils.data import Dataset, DataLoader
import torch

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import f1_score, classification_report, confusion_matrix

import gc
import time
import random
import os
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 필요한 함수 정의

In [5]:
def seed_everything(seed:int):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

    
# 텍스트 전처리. 
# ip 패턴(ex 127.0.0.1) 혹은 ip.port(ex 127.0.0.1.8888) 를 모두 "IP"로 치환.
# 영어와 한글만 남김ㅅ
def text_preprocessing(texts):
    result=[]
    for log in tqdm(texts):
        log = log.replace('\\', '')
        log = re.sub(r"\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}:\d{1,5}\b", 'IP', log)
        log = re.sub(r"\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b", 'IP', log)
        eng_kor = re.compile('[^a-zA-Zㄱ-ㅣ가-힣]+') 
        log = eng_kor.sub(' ', log)
        result.append(log)
    return result

# vocabulary 생성 함수
# 문장을 space단위로 분리 후 vocab을 생성
def return_vocab(texts):
    vocab = set()
    for text in tqdm(texts):
        words = text.split()
        for word in words:
            vocab.add(word)
    vocab = list(vocab)
    vocab = {word:i for word,i in zip(vocab, range(len(vocab)))}

    vocab_count = {w:0 for w in vocab}

    for text in tqdm(texts):
        words = text.split()
        for word in words:
            vocab_count[word]+=1

    vocab = {}
    idx=0
    for word, cnt in tqdm(zip(vocab_count.keys(), vocab_count.values())):
        if cnt>=1:
            vocab[word] = idx+1
            idx+=1
    return vocab


# word 단위 token을 정수로 치환하여 정수 시퀀스를 만듦
def word2int(li, vocab, train=False):
    result=[]
    for text in tqdm(li):
        tmp = []
        text = text.split()
        for word in text:
            if word in vocab.keys():
                tmp.append(vocab[word]+1)
        result.append(tmp)
    return result


# 같은 길이로 만들어 주기 위해 뒷부분을 padding(0)으로 채움
# max_length(최대 길이)는 해당 노트북에서는 200으로 함.
def padding(sequences, max_length):
    result=[]
    for sequence in tqdm(sequences):
        if len(sequence) < max_length:
            sequence +=[0]*(max_length-len(sequence))
        else:
            sequence = sequence[:max_length]
        result.append(sequence)
    return np.array(result)

# 텍스트 유사도 계산
def return_similarity(a, b): 
    c = a.intersection(b)
    return float(len(c))/(len(a)+len(b)-len(c))



def softmax(k):
    exp_k = np.exp(k)
    sum_exp_k = np.sum(exp_k)
    y = exp_k / sum_exp_k
    return y


# threshold 값 이상인 값이 없으면 7로 반환
def level(array, threshold):
    if (array>=threshold).any() == True:
        return array.argmax()
    else:
        return 7
    

def return_level_array(array, threshold, count_print=False):
    result = np.array([level(k,threshold) for k in array])
    if count_print==True:
        print(f'threshold : {threshold}              ',np.unique(result, return_counts=True)[1])
    return result


def model_save(model, path):
    torch.save({
        'model': model.state_dict(),
    }, path)

# 전처리
- 처음 데이터를 볼 당시 비 정상적으로 긴 길이의 log가 나와서 452419를 제외 --> 영어와 한글만 남기는 전처리를 거치면 사용해도 상관없을것으로 보임
- 치환, 영어와 한글만 남기는 전처리 후 중복되는 텍스트는 하나만 남김. 
- level 2,4,6은 샘플 수가 적기 때문에 중복되는 텍스트라도 제거하지 않고 validation 성능 측정 때 사용되도록 함.

In [13]:
dataset = pd.read_csv('log_data_clean.csv')
dataset = dataset.drop('Unnamed: 0', axis = 1)
dataset.head()

,user_id,event,timestamp,date_cd
0,576409,StartLoanApply,2022-03-25 11:12:09,2022-03-25
1,576409,ViewLoanApplyIntro,2022-03-25 11:12:09,2022-03-25
2,72878,EndLoanApply,2022-03-25 11:14:44,2022-03-25
3,645317,OpenApp,2022-03-25 11:15:09,2022-03-25
4,645317,UseLoanManage,2022-03-25 11:15:11,2022-03-25


In [14]:
dataset = pd.read_csv('log_data_clean.csv')
dataset = dataset.drop('Unnamed: 0', axis = 1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset[['user_id','timestamp',
       'date_cd']], dataset['event'])

train = pd.concat([X_train, y_train], axis = 1)
test = pd.concat([X_test, y_test], axis= 1)

## 텍스트 유사도
- 숫자를 제외한 영어와 한글만 남겨 전처리 했을 때 한 단어 차이로 level이 다른 샘플이 존재함.
- 제공된 데이터 전처리한 결과들 끼리 자카드 유사도를 구하여 유사도가 0.5 이상인데 level이 다르면 무조건 훈련샘플로 사용하기 위해 해당 index를 저장해둠.
- 60818 * 60818 번 loop 가 실행되기 때문에 시간이 오래걸림

## 훈련/검증  데이터 분리
- 5-fold로 분리
- 각 폴드에서 텍스트 유사도가 높던것은 훈련 샘플로 넣고 검증 샘플로 들어간것은 제거.


In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in skf.split(TRAIN, Y_train):
    train_idx = np.array(list(set(list(train_idx)+list(sim_idx))))
    valid_idx = np.array(list(set(set(valid_idx)-set(sim_idx))))
    folds.append((train_idx, valid_idx))

# DataLoader
- 로더에서는 다른 처리는 없고 배치단위로 불러오기만 합니다.

In [6]:
class LogDataset(Dataset):
    def __init__(self, X, y=None, train=True):
        self.X = X
        self.y = y
        self.train=train
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        X = self.X[idx]
        if self.train==True:
            y = self.y[idx]
            return X, y
        else:
            return X


# Multiple-Kernel Size CNNs-BiLSTM
#### 각각 다른 kernel size를 가진 Conv를 통해 시퀀스에서 패턴을 추출하고 LSTM을 통해 순서정보를 학습하는 것을 기대하고 모델을 구성하였습니다.

- 전처리된 정수 시퀀스가 모델에 입력.(x)
- 입력된 정수 시퀀스는 Embedding Layer를 통해 각 정수들은 1024 차원으로 인코딩되어 벡터 시퀀스로 변환.(nn.Embedding)
- 벡터 시퀀스는 kernel size가 다른 6개의 Conv로 입력(nn.Conv1d)되고 출력된 feature map은 Maxpooling.(nn.MaxPool1d)
- 6개의 CNN에서 출력된 값들을 이어준 후 (torch.cat) BatchNorm을 적용.
- BiLSTM에 입력(nn.LSTM)되고 출력된 값들을 average pooling 한후 squeeze 해줌.
- 마지막 출력은 class 개수(7)로 출력.



In [7]:
class Log_classification(nn.Module):
    def __init__(self, b=None):
        super(Log_classification, self).__init__()
        
        # Embedding
        self.embedding = nn.Embedding(len(vocab)+3, embedding_dim=1024, padding_idx=0)
        
        # Conv
        self.conv1 = nn.Conv1d(in_channels=1024, out_channels=512, kernel_size=2, padding=1)
        self.conv2 = nn.Conv1d(in_channels=1024, out_channels=512, kernel_size=4, padding=2)
        self.conv3 = nn.Conv1d(in_channels=1024, out_channels=512, kernel_size=6, padding=3)
        self.conv4 = nn.Conv1d(in_channels=1024, out_channels=512, kernel_size=8, padding=4)
        self.conv5 = nn.Conv1d(in_channels=1024, out_channels=512, kernel_size=10, padding=5)
        self.conv6 = nn.Conv1d(in_channels=1024, out_channels=512, kernel_size=12, padding=6)
        
        # LSTM
        self.lstm1 = nn.LSTM(input_size=100, hidden_size=32, bidirectional=True, num_layers=1, dropout=0.3)
        
        # Pooling
        self.maxpooling = nn.MaxPool1d(2)
        self.average_pooling = nn.AdaptiveAvgPool1d(1)
        
        # BatchNormalization
        self.BN1 = nn.BatchNorm1d(512*6)
        
        # Dropout
        self.dropout = nn.Dropout(0.15)
        
        # Output
        self.output = nn.Linear(512*6, 7)
        
        
    def forward(self, x, return_mid=False):
        x_ = self.embedding(x)
        x_ = torch.movedim(x_, 1, 2)
        
        
        conv_k1_x = F.elu(self.conv1(x_))
        conv_k1_x = self.maxpooling(conv_k1_x)
        
        conv_k2_x = F.elu(self.conv2(x_))
        conv_k2_x = self.maxpooling(conv_k2_x)
        
        conv_k3_x = F.elu(self.conv3(x_))
        conv_k3_x = self.maxpooling(conv_k3_x)
        
        conv_k4_x = F.elu(self.conv4(x_))
        conv_k4_x = self.maxpooling(conv_k4_x)
        
        conv_k5_x = F.elu(self.conv5(x_))
        conv_k5_x = self.maxpooling(conv_k5_x)
        
        conv_k6_x = F.elu(self.conv6(x_))
        conv_k6_x = self.maxpooling(conv_k6_x)
        
        
        
        conv_x = torch.cat([conv_k1_x, conv_k2_x, conv_k3_x, conv_k4_x, conv_k5_x, conv_k6_x], 1)
        conv_x_ = self.BN1(conv_x)
        conv_x = self.dropout(conv_x_)
        
        lstm, _ = self.lstm1(conv_x)
        lstm = self.average_pooling(lstm)
        lstm = torch.squeeze(lstm)
        return_value = lstm

        output = (self.output(lstm))
        if return_mid == True:
            return output, return_value
        else:
            return output


# Training
- 5개 폴드
- Epoch : 30
- batchSize : 400
- optimizer : adam
- scheduler : CosineAnnealingLR
- loss : CrossEntropy
- save : validation f1-macro
- 훈련시 confidence가 0.85 이상인 것이 없으면 7로 분류하여 성능평가하였음.


In [9]:
seed_everything(random_state)

for fold in range(5):
    save_dir = 'model'
    model_name = f'max_length({max_length})-random_state({random_state})-fold({fold+1})'
    if fold==0:
        with open(f'vocab/{model_name}.bin', 'wb') as f:
            pickle.dump(vocab, f)

    epochs=30
    batch_size=400
    best = 0
    train_idx, valid_idx = folds[fold]

    train_dataset = LogDataset(TRAIN[train_idx], Y_train[train_idx])
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    
    valid_dataset = LogDataset(TRAIN[valid_idx], Y_train[valid_idx])
    valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=False)

    
    # Build model
    model = Log_classification().to(device)
    model = nn.DataParallel(model, device_ids=[0,1,2])

    
    # Optimizer
    Q = math.floor(len(train_dataset)/batch_size+1)*epochs/7
    optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
    lrs = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = Q)
    
    
    # Loss
    criterion = nn.CrossEntropyLoss()


    for epoch in range(epochs):
        start=time.time()
        
        ####################################################### TRAIN
        model.train()
        train_loss=0
        train_pred=[]
        train_true=[]

        for X, y in (train_loader):
            X = torch.tensor(X, dtype=torch.long, device=device)
            y = torch.tensor(y, dtype=torch.long, device=device)

            optimizer.zero_grad()
            pred = model(X)
            loss = criterion(pred, y)
            loss.backward()
            optimizer.step()
            lrs.step()

            train_loss += loss.item()
            train_pred += list(pred.detach().cpu().numpy())
            train_true += list(y.detach().cpu().numpy())
        
        
        train_pred = np.array([level(softmax(k), 0.85) for k in train_pred])
        train_f1_macro = f1_score(train_true, train_pred, average='macro', labels=[0,1,2,3,4,5,6])
        train_loss = train_loss/len(train_loader)

        ####################################################### VALIDATION
        model.eval()
        valid_loss=0
        valid_pred=[]
        valid_true=[]

        for X, y in (valid_loader):
            X = torch.tensor(X, dtype=torch.long, device=device)
            y = torch.tensor(y, dtype=torch.long, device=device)

            pred = model(X)
            loss = criterion(pred, y)
            
            
            valid_loss += loss.item()
            valid_pred += list(pred.detach().cpu().numpy())
            valid_true += list(y.detach().cpu().numpy())
        
        
        valid_pred = np.array([level(softmax(k), 0.85) for k in valid_pred])        
        valid_f1_macro = f1_score(valid_true, valid_pred, average='macro', labels=[0,1,2,3,4,5,6])
        valid_loss = valid_loss/len(valid_loader)

        
        
        running_time=time.time()-start
        print(f'Epoch : {epoch+1}/{epochs}    TRAIN -> loss : {train_loss:.4f}  f1-macro : {train_f1_macro}    //    VALID -> loss : {valid_loss:.4f}  f1-macro : {valid_f1_macro}  time : {running_time:.0f}/{(epochs-epoch)*running_time:.0f}')
        print(classification_report(valid_true, valid_pred, labels=[0,1,2,3,4,5,6]))
        print(confusion_matrix(valid_true, valid_pred, labels=[0,1,2,3,4,5,6,7]))
        if valid_f1_macro>=best:
            best = valid_f1_macro
            model_save(model=model, path=f'{save_dir}/{model_name}.pt')

Epoch : 1/30    TRAIN -> loss : 0.0892  f1-macro : 0.37727993427702966    //    VALID -> loss : 0.0133  f1-macro : 0.5286556202378457  time : 45/1336
              precision    recall  f1-score   support

           0       1.00      0.86      0.93       770
           1       1.00      1.00      1.00     10928
           2       0.00      0.00      0.00         2
           3       1.00      0.74      0.85        70
           4       0.00      0.00      0.00         2
           5       1.00      0.86      0.92       114
           6       0.00      0.00      0.00         2

   micro avg       1.00      0.98      0.99     11888
   macro avg       0.57      0.49      0.53     11888
weighted avg       1.00      0.98      0.99     11888

[[  664     0     0     0     0     0     0   106]
 [    1 10888     0     0     0     0     0    39]
 [    0     0     0     0     0     0     0     2]
 [    0     0     0    52     0     0     0    18]
 [    0     0     0     0     0     0     0     2

Epoch : 9/30    TRAIN -> loss : 0.0073  f1-macro : 0.9348055807043478    //    VALID -> loss : 0.0111  f1-macro : 0.9882816635739403  time : 38/834
              precision    recall  f1-score   support

           0       0.98      0.93      0.95       770
           1       1.00      1.00      1.00     10928
           2       1.00      1.00      1.00         2
           3       1.00      0.96      0.98        70
           4       1.00      1.00      1.00         2
           5       1.00      0.97      0.99       114
           6       1.00      1.00      1.00         2

   micro avg       1.00      0.99      1.00     11888
   macro avg       1.00      0.98      0.99     11888
weighted avg       1.00      0.99      1.00     11888

[[  713     8     0     0     0     0     0    49]
 [    8 10908     0     0     0     0     0    12]
 [    0     0     2     0     0     0     0     0]
 [    1     0     0    67     0     0     0     2]
 [    0     0     0     0     2     0     0     0]


Epoch : 17/30    TRAIN -> loss : 0.0031  f1-macro : 0.9905829326193193    //    VALID -> loss : 0.0141  f1-macro : 0.9904645686003507  time : 38/531
              precision    recall  f1-score   support

           0       0.98      0.93      0.96       770
           1       1.00      1.00      1.00     10928
           2       1.00      1.00      1.00         2
           3       1.00      0.99      0.99        70
           4       1.00      1.00      1.00         2
           5       1.00      0.97      0.99       114
           6       1.00      1.00      1.00         2

   micro avg       1.00      0.99      1.00     11888
   macro avg       1.00      0.98      0.99     11888
weighted avg       1.00      0.99      1.00     11888

[[  715    15     0     0     0     0     0    40]
 [    8 10911     0     0     0     0     0     9]
 [    0     0     2     0     0     0     0     0]
 [    1     0     0    69     0     0     0     0]
 [    0     0     0     0     2     0     0     0]

Epoch : 25/30    TRAIN -> loss : 0.0007  f1-macro : 0.9854641130319836    //    VALID -> loss : 0.0183  f1-macro : 0.9905343578973852  time : 38/227
              precision    recall  f1-score   support

           0       0.99      0.93      0.96       770
           1       1.00      1.00      1.00     10928
           2       1.00      1.00      1.00         2
           3       1.00      0.99      0.99        70
           4       1.00      1.00      1.00         2
           5       1.00      0.97      0.99       114
           6       1.00      1.00      1.00         2

   micro avg       1.00      0.99      1.00     11888
   macro avg       1.00      0.98      0.99     11888
weighted avg       1.00      0.99      1.00     11888

[[  714    20     0     0     0     0     0    36]
 [    6 10914     0     0     0     0     0     8]
 [    0     0     2     0     0     0     0     0]
 [    1     0     0    69     0     0     0     0]
 [    0     0     0     0     2     0     0     0]

# level7-validation

In [8]:
weights = sorted(glob('model/*'))

In [29]:
with open(f'vocab/max_length({max_length})-random_state({random_state})-fold(1).bin', 'rb') as f:
    vocab = pickle.load(f)

weights = sorted(glob('model/*'))
level7_df = pd.read_csv('dataset/validation_sample.csv')


level7_preds = 0
for w in weights:
    
    model=Log_classification().to(device)
    model = nn.DataParallel(model, device_ids=[0])
    
    
    weight = torch.load(w)
    model.load_state_dict(weight['model'])
    model.eval()

    level7=pd.concat([level7_df])
    level7_log = text_preprocessing(level7['full_log'].values)
    level7_log = word2int(level7_log, vocab)
    level7_log = padding(level7_log, max_length=max_length)
    level7_log = torch.tensor(level7_log, dtype=torch.long, device=device)
    level7_log = model(level7_log).detach().cpu().numpy()
    level7_log = level7_log[:3]
    level7_log = (np.array([softmax(k) for k in level7_log]).round(3))
    level7_preds += level7_log/5
    

100%|██████████| 3/3 [00:00<00:00, 34473.73it/s]


# Test inference

- 훈련 데이터와 같은 방법으로 전처리. vocab은 훈련데이터에서 만든 vocab을 사용
- 5-fold 훈련을 했지만 validation 성능이 좋지 않았던 4번 째 fold로 만든 모델은 test inference로 사용하지 않음.
- 4개 모델로 예측한 값들을 평균내어 confidence가 0.7 이상인 것이 없으면 level 7로 반환 하였음.

In [10]:
with open(f'vocab/max_length({max_length})-random_state({random_state})-fold(1).bin', 'rb') as f:
    vocab = pickle.load(f)

test = pd.read_csv('dataset/test.csv')
test_log = text_preprocessing(test['full_log'].values)
TEST = word2int(test_log, vocab, train=True)
TEST = padding(TEST, max_length=max_length)


100%|██████████| 1418916/1418916 [00:02<00:00, 560883.78it/s]


In [11]:
def output(model_name):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    save_dir = 'model'
    model = Log_classification().to(device)
    model = nn.DataParallel(model, device_ids=[0,1,2])
    weights = torch.load(model_name)
    
    model.load_state_dict(weights['model'])
    model.eval()

    batch_size=600
    test_dataset = LogDataset(TEST, None, train=False)
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

    test_pred=[]
    for X in tqdm(test_loader):
        X = torch.tensor(X, dtype=torch.long, device=device)
        pred = model(X)
        test_pred += list(pred.detach().cpu().numpy())


    test_pred = np.array([softmax(k) for k in test_pred])
    return np.array(test_pred)

In [12]:
weights = sorted(glob('model/*'))
test_preds = []

for w in weights:
    test_preds.append(output(w))
test_preds = np.array(test_preds)

100%|██████████| 2365/2365 [05:23<00:00,  7.31it/s]


In [23]:
submit_level = return_level_array((test_preds[0]+test_preds[1]+test_preds[2]+test_preds[4])/4, 0.7, count_print=True)

threshold : 0.7               [1002152  396509      34   12958      34    6535      26     668]


In [26]:
submission = pd.read_csv('dataset/sample_submission.csv')
submission['level'] = submit_level
submission

,id,level
0,1000000,0
1,1000001,0
2,1000002,1
3,1000003,0
4,1000004,1
...,...,...
1418911,2418911,0
1418912,2418912,0
1418913,2418913,1
1418914,2418914,0


In [27]:
submission.to_csv('submission.csv', index=False)